In [18]:
import pandas as pd
import sqlite3

In [19]:
#Function for returning query results.
def result(q):
    with sqlite3.connect("C:/PythP/factbook_expanded.db") as conn:
        return pd.read_sql_query(q,conn)
    
#Function for returning information about the database.  
def check_master():
    with sqlite3.connect("C:/PythP/factbook_expanded.db") as conn:
        qr = "SELECT * FROM sqlite_master WHERE type IN ('table','view');"
        return result(qr)
    
pd.set_option('display.max_columns', None)

In [20]:
check_master()

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,facts,facts,47,"CREATE TABLE ""facts"" (""id"" INTEGER PRIMARY KEY..."
2,table,cities,cities,2,CREATE TABLE cities (\n id integer prim...


# Quick revision using a single table

In [21]:
facts_prev = """SELECT * 
FROM facts
LIMIT 10;
"""
result(facts_prev)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate
0,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51
1,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30
2,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92
3,4,an,Andorra,468,468,0,85580,0.12,8.13,6.96,0.00
4,5,ao,Angola,1246700,1246700,0,19625353,2.78,38.78,11.49,0.46
5,6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21
6,7,ar,Argentina,2780400,2736690,43710,43431886,0.93,16.64,7.33,0.00
7,8,am,Armenia,29743,28203,1540,3056382,0.15,13.61,9.34,5.80
8,9,as,Australia,7741220,7682300,58920,22751014,1.07,12.15,7.14,5.65
9,10,au,Austria,83871,82445,1426,8665550,0.55,9.41,9.42,5.56


***Return the countries with:-***

1) Maxiumum and minimum population.

2) Maximum and minimum population growth.

In [22]:
max_pop = """
SELECT 
    name,
    population
FROM facts
WHERE population = (SELECT MAX(population) FROM facts);"""

min_pop = """
SELECT 
    name,
    population
FROM facts
WHERE population = (SELECT MIN(population) FROM facts);"""

max_pop_growth = """
SELECT 
    name,
    population_growth
FROM facts
WHERE population_growth = (SELECT MAX(population_growth) FROM facts);"""

min_pop_growth = """
SELECT 
    name,
    population_growth
FROM facts
WHERE population_growth = (SELECT MIN(population_growth) FROM facts);"""

result(max_pop)

,name,population
0,World,7256490011


Looks like the table has erronious data where the entire world's population is registered as a country. 

In [23]:
result(min_pop)

,name,population
0,Antarctica,0


In [24]:
result(max_pop_growth)

,name,population_growth
0,South Sudan,4.02


In [25]:
result(min_pop_growth)

,name,population_growth
0,Holy See (Vatican City),0.0
1,Cocos (Keeling) Islands,0.0
2,Greenland,0.0
3,Pitcairn Islands,0.0


***The top 10 countries with a population greater than 150000 having the highest and lowest land to population ratio.*** 

In [26]:
high_land = """
SELECT 
    name as "Country"
from facts
WHERE population > 150000
ORDER BY CAST(area as Float)/population DESC
LIMIT 10;
"""
result(high_land)

,Country
0,Mongolia
1,Western Sahara
2,Namibia
3,Australia
4,Iceland
5,Guyana
6,Mauritania
7,Canada
8,Suriname
9,Libya


In [27]:
low_land = """
SELECT 
    name as "Country"
from facts
WHERE population > 150000
ORDER BY CAST(area as Float)/population ASC
LIMIT 10;
"""

result(low_land)

,Country
0,Chad
1,Niger
2,World
3,Macau
4,Singapore
5,Hong Kong
6,Gaza Strip
7,Bahrain
8,Maldives
9,Malta


# Using Joins to combine facts and cities table

In [28]:
cities_check = """
SELECT *
FROM cities
LIMIT 10;"""

result(cities_check)

,id,name,population,capital,facts_id
0,1,Oranjestad,37000,1,216
1,2,Saint John'S,27000,1,6
2,3,Abu Dhabi,942000,1,184
3,4,Dubai,1978000,0,184
4,5,Sharjah,983000,0,184
5,6,Kabul,3097000,1,1
6,7,Algiers,2916000,1,3
7,8,Oran,783000,0,3
8,9,Baku,2123000,1,11
9,10,Tirana,419000,1,2


The facts_id column is the foreign key here.

- `id` - A unique ID for each city.
- `name` - The name of the city.
- `population` - The population of the city.
- `capital` - Whether the city is a capital city: 1 if it is, 0 if it isn't.
- `facts_id` - The ID of the country, from the facts table.

Note: The data is not included for all countries. Furthermore, even if a country does have a city value, it does not mean that it is the only city(s) in that country. 

Starting simple ...

***Return 15 name columns from facts and cities table.***

In [29]:
all_country_city = """
SELECT 
    cnt.name as "Country",
    ct.name as "City"
FROM facts as cnt
INNER JOIN cities as ct
ON cnt.id = ct.facts_id
LIMIT 15;"""

result(all_country_city)

,Country,City
0,Aruba,Oranjestad
1,Antigua and Barbuda,Saint John'S
2,United Arab Emirates,Abu Dhabi
3,United Arab Emirates,Dubai
4,United Arab Emirates,Sharjah
5,Afghanistan,Kabul
6,Algeria,Algiers
7,Algeria,Oran
8,Azerbaijan,Baku
9,Albania,Tirana


***Join cities to facts using INNER JOIN and return all columns from 'facts' and only city name from 'cities'. Limit result to 10 rows***

In [30]:
all_again = """
    SELECT cnt.*,
    ct.name as "City Name"
FROM cities as ct
INNER JOIN facts as cnt
ON ct.facts_id = cnt.id
LIMIT 10;"""

result(all_again)

,id,code,name,area,area_land,area_water,population,population_growth,birth_rate,death_rate,migration_rate,City Name
0,216,aa,Aruba,180,180,0,112162,1.33,12.56,8.18,8.92,Oranjestad
1,6,ac,Antigua and Barbuda,442,442,0,92436,1.24,15.85,5.69,2.21,Saint John'S
2,184,ae,United Arab Emirates,83600,83600,0,5779760,2.58,15.43,1.97,12.36,Abu Dhabi
3,184,ae,United Arab Emirates,83600,83600,0,5779760,2.58,15.43,1.97,12.36,Dubai
4,184,ae,United Arab Emirates,83600,83600,0,5779760,2.58,15.43,1.97,12.36,Sharjah
5,1,af,Afghanistan,652230,652230,0,32564342,2.32,38.57,13.89,1.51,Kabul
6,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92,Algiers
7,3,ag,Algeria,2381741,2381741,0,39542166,1.84,23.67,4.31,0.92,Oran
8,11,aj,Azerbaijan,86600,82629,3971,9780780,0.96,16.64,7.07,0.00,Baku
9,2,al,Albania,28748,27398,1350,3029278,0.30,12.92,6.58,3.30,Tirana


***Return each country and its capital city for the top 10 cities with the highest population.***

In [31]:
cap_low_mig = """
SELECT 
    ft.name as "Country",
    ct.name as "Capital city"
FROM facts as ft
INNER JOIN cities as ct
ON ft.id = ct.facts_id
WHERE ct.capital = 1
ORDER BY ft.migration_rate
LIMIT 10;
"""

result(cap_low_mig)

,Country,Capital city
0,Falkland Islands (Islas Malvinas),Stanley
1,Montenegro,Podgorica
2,Azerbaijan,Baku
3,Andorra,Andorra La Vella
4,Argentina,Buenos Aires
5,"Bahamas, The",Nassau
6,Belize,Belmopan
7,Benin,Porto-Novo
8,Bhutan,Thimphu
9,Burundi,Bujumbura


***Return 10 countries for which there are no corresponding city values in 'cities'***

In [32]:
leftj_ftct = """
SELECT 
    ft.name as "Country",
    ct.name as "City" 
FROM facts as ft
LEFT JOIN cities as ct
ON ft.id = ct.facts_id
WHERE "City" IS NULL
LIMIT 20;
"""
result(leftj_ftct)

,Country,City
0,Kosovo,None
1,Monaco,None
2,Nauru,None
3,San Marino,None
4,Singapore,None
5,Holy See (Vatican City),None
6,Taiwan,None
7,European Union,None
8,Ashmore and Cartier Islands,None
9,Christmas Island,None


***Return Country, Capital city, and city population for all capital cities that have a population greater than 10,000,000. Limit to 5 rows and order by highest to lowest city population.***

In [33]:
#Using sub queries to mix it up eventhough we may use a simple where condition ....

pop_cities = """
SELECT 
    ft.name as "Country",
    tct.name as "City",
    tct.population as "City population"
FROM facts as ft
INNER JOIN 
    (SELECT 
        name,
        population,
        facts_id
    FROM cities 
    WHERE population > 10000000 AND capital = 1) 
    AS tct
ON ft.id = tct.facts_id
ORDER BY 3 DESC
LIMIT 5;"""

result(pop_cities)

,Country,City,City population
0,Japan,Tokyo,37217000
1,India,New Delhi,22654000
2,Mexico,Mexico City,20446000
3,China,Beijing,15594000
4,Bangladesh,Dhaka,15391000


***Return the following columns, in order:-***

- Country, the name of the country.
- urban_pop, the sum of the population in major urban areas (citites) belonging to that country.
- total_pop, the total population of the country.
- urban_pct, the percentage of the popularion within urban areas, calculated by dividing urban_pop by total_pop, multiplied by 100.

Only countries that have an urban_pct greater than 0.5.

Rows should be sorted by urban_pct in ascending order and limited to 20 results.

In [34]:
urban = """
SELECT 
    ft.name as "Country",
    SUM(ct.population) as urban_pop,
    ft.population as "total_pop",
    ROUND((CAST(SUM(ct.population) as float)/ft.population) * 100,3) || '%' as "urban_pct" 
FROM facts as ft
INNER JOIN cities as ct
ON ft.id = ct.facts_id
GROUP BY 1
HAVING "urban_pct" > 0.5
ORDER BY 4
LIMIT 20;
"""
result(urban)

,Country,urban_pop,total_pop,urban_pct
0,Cambodia,155000,15708756,0.987%
1,Austria,172000,8665550,1.985%
2,Bosnia and Herzegovina,389000,3867055,10.059%
3,Zimbabwe,1542000,14229541,10.837%
4,Burkina Faso,2053000,18931686,10.844%
5,Eritrea,712000,6527689,10.907%
6,Solomon Islands,68000,622469,10.924%
7,Guam,169000,161785,104.46%
8,Northern Mariana Islands,56000,52344,106.985%
9,Mauritius,151000,1339827,11.27%
